# Figure 4.5 of Chapter 4

In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
total_population = 1e5
latent_period = 8.
infectious_period = 7.
r0 = 13.
parameters = {
    "beta": r0 / infectious_period,
    "progression": 1. / latent_period,
    "recovery_rate": 1. / infectious_period,
}
infectious_seed = 1.  # Parameter not stated

In [ ]:
def build_seir_model(prop_recovered):
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    seir_model = CompartmentalModel(
        times=(0., 2000.),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    seir_model.set_initial_population(
        distribution={
            "Susceptible": total_population * (1. - prop_recovered) - infectious_seed, 
            "Infectious": infectious_seed,
            "Immune": total_population * prop_recovered,
        }
    )
    seir_model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("beta"),
        source="Susceptible",
        dest="Pre-infectious"
    )
    seir_model.add_transition_flow(
        name="progression", 
        fractional_rate=Parameter("progression"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    seir_model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="Infectious", 
        dest="Immune"
    )
    seir_model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    return seir_model

In [ ]:
outputs = pd.DataFrame()
immune_props = (0., 0.9, 0.92, 0.923, 0.93, 0.95,)
for immune_prop in immune_props:
    seir_model = build_seir_model(immune_prop)
    seir_model.run(parameters=parameters)
    outputs[immune_prop] = seir_model.get_derived_outputs_df()["incidence"]

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

for prop in immune_props:
    left_ax.plot(
        seir_model.times, 
        outputs[prop],
        label=prop,
    )
left_ax.set_yscale('log')
left_ax.set_ylim((0.01, 1e4))
left_ax.set_xlim((0., 2000.))
left_ax.set_ylabel("Number of infectious individuals/100,000/day")
left_ax.set_title("Measles")
left_ax.legend()

image = plt.imread("./figures/fig_4_5.jpg")
right_ax.imshow(image, aspect="auto")
right_ax.axis("off");